In [ ]:
import pandas as pd
from neuralforecast.core import NeuralForecast
from neuralforecast.models import TimesNet
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd

# Загрузка файла и преобразование столбца date к формату datetime
df = pd.read_csv('dataset_bar_harmonics_median.csv', parse_dates=['date'])

# Отсечение данных после 1 июля 2025 включительно
cutoff_date = pd.Timestamp('2025-07-01')
filtered_df = df[df['date'] <= cutoff_date]

# Сохранение или дальнейшая работа
filtered_df.to_csv('dataset_bar_harmonics_median_filtered.csv', index=False)


In [ ]:
# Загрузка вашего файла
df = pd.read_csv('dataset_bar_harmonics_median_filtered.csv', parse_dates=['date'])

# Преобразуем к нужному формату
df = df.rename(columns={'date': 'ds', 'value': 'y'})
df['unique_id'] = 'id_1'  # один временной ряд

# Проверка
print(df.head())


In [ ]:
horizon = 200          # горизонт прогноза (выберите под свою задачу)
input_size = 900       # размер окна истории (обычно в 2–4 раза больше горизонта)

In [ ]:

# Инициализация модели
model = TimesNet(
    h=horizon,
    input_size=input_size,
    max_steps=100,
    learning_rate=1e-3,
    scaler_type='standard'
)

# Определение контейнера моделей (важно указать freq вашей серии)
nf = NeuralForecast(models=[model], freq='D')  # или 'H', если данные почасовые

# Обучение модели
nf.fit(df=df)


In [ ]:
forecast_df = nf.predict()

In [ ]:
actual = df.set_index('ds')['y']
forecast = forecast_df.set_index('ds')['TimesNet']

plt.figure(figsize=(12, 6))
plt.scatter(actual.index, actual.values, color='black', label='Исторические данные', s=10)
plt.scatter(forecast.index, forecast.values, color='blue', label='TimesNet прогноз', s=10)
plt.xlabel('Дата')
plt.ylabel('Значение')
plt.title('Исторические данные и прогноз TimesNet (только точки)')
plt.legend()
plt.grid(True)
plt.show()